In [16]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [10]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [3]:
#Import data from the processing of data in the journal_training_word2vec notebook
#df = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/C_article_replication_and_translational_classification/df.p', 'rb'))
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/pickled_objects/dfJournals_trans_categories.p', 'rb'))

In [4]:
print df.shape
print df.columns.values
print df['sentiment'].unique()

(97048, 5)
['sentiment' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']
['T0' 'T1/T2' 'T3/T4']


In [5]:
#for the journals VM dataframe, change the label of the 'sentiment' column to 'label'
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

In [6]:
df[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,


In [18]:
df_val = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/df.p', 'rb'))

### Apply Word2Vec Inversion to get sentence scores

In [15]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprobsentences(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    #prob = prob.groupby("doc").mean()
    return prob

In [24]:
import numpy as np

def getFeatures(probssentences_train, train1):
    features = np.zeros((np.unique(probssentences_train["doc"]).shape[0],11))
    label = []
    for doc in np.unique(probssentences_train["doc"]):
        inx = np.where(probssentences_train["doc"] == doc)[0]
        subset = probssentences_train[0].iloc[inx]
        score = train1['boolean_label'].iloc[doc]
        label.append(score)

        inxFirst = inx[0]
        inxLast = inx[-1]

        features[doc,0] = np.mean(subset) #mean of scores
        features[doc,1] = np.min(subset) #min score
        features[doc,2] = np.max(subset) #max score
        features[doc,3] = np.std(subset) #stdev of scores
        features[doc,4] = probssentences_train[0].iloc[inxFirst] #score of first sentence
        features[doc,5] = probssentences_train[0].iloc[inxLast] #score of last sentences
        features[doc,6] = len(subset) #number of sentences
        features[doc,7] = np.mean(                               #sigmoid (with k = 10)
                            1./(1+np.exp(-10*
                               subset.astype(float)-0.5 ))) 
        features[doc,8] = np.mean(                               #sigmoid (with k = 20)
                            1./(1+np.exp(-20*
                               subset.astype(float)-0.5 )))
        features[doc,9] = np.argmin(subset)-inxFirst #position of min score
        features[doc,10] = np.argmax(subset)-inxFirst #position of max score
    
    return (features, label)

In [25]:
def buildModels(train1, test1, docs, docsTrain, df_val, docs_val, basemodel, label):
    
    #add boolean labels
    train1['boolean_label'] = 1
    for row in train1.iterrows():
        if row[1]['label'] != label:
            train1.set_value(row[0], 'boolean_label', 0)
            
    test1['boolean_label'] = 1
    for row in test1.iterrows():
        if row[1]['label'] != label:
            test1.set_value(row[0], 'boolean_label', 0)
            
    df_val['boolean_label'] = 1
    for row in df_val.iterrows():
        if row[1]['label'] != label:
            df_val.set_value(row[0], 'boolean_label', 0)
    
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    inxs_pos = np.where(train1["boolean_label"] == 1)[0].tolist()
    inxs_neg = np.where(train1["boolean_label"] == 0)[0].tolist()

    print "Parsing sentences from training set"
    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

    models = [deepcopy(basemodel) for i in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )

    print "scoring test set"
    # get the probs (note we give docprob our test set plus the models)
    probssentencesArticle = docprobsentences(docs,models).astype(object)

    probssentences_train = docprobsentences(docsTrain,models).astype(object)
    
    probssentences_val = docprobsentences(docs_val, models).astype(object)
    
    #step 1- get training features
    obj = getFeatures(probssentences_train, train1)
    features = obj[0]
    label = obj[1]

    #step 2- build decision tree
    clf = tree.DecisionTreeClassifier(min_samples_leaf=1000)
    #criterion = 'entropy
    clf = clf.fit(features, label)

    print "decision tree performance on self"
    resultTrain = clf.predict(features)

    print roc_auc_score(label, resultTrain)

    #step 3- build random forest

    forest = RandomForestClassifier()
    forest = forest.fit( features, label )

    print "random forest performance on self"
    resultTrainRF = forest.predict(features)
    print roc_auc_score(label, resultTrainRF)

    #step 4- get test features
    test = getFeatures(probssentencesArticle, test1)
    featuresT = test[0]
    labelsT = test[1]
    
    #step 4.5- get validation features
    val = getFeatures(probssentences_val, df_val)
    featuresV = val[0]
    labelsV = val[1]

    #step 5- get test set performance on decision tree 
    result = clf.predict(featuresT)
    scoreDT = roc_auc_score(labelsT, result)

    #step 6- get test set performance on random forest
    clf2 = forest.predict(featuresT)
    scoreRF = roc_auc_score(labelsT, clf2)
    
    #step 7- decision tree performance on validation set
    result3 = clf.predict(featuresV)
    scoreDT_val = roc_auc_score(labelsV, result3)
    
    #step 8- random forest performance on validation set
    clf3 = forest.predict(featuresV)
    scoreRF_val = roc_auc_score(labelsV, clf3)
    
    return scoreDT, scoreRF, models, probssentencesArticle, probssentences_train, clf, forest, scoreDT_val, scoreRF_val

In [ ]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=10)

i = 1
avgDT_All = []
avgDT_T0 = []
avgDT_T1T2 = []
avgDT_T3T4 = []
avgRF_All = []
avgRF_T0 = []
avgRF_T1T2 = []
avgRF_T3T4 = []

avgDT_All_val = []
avgDT_T0_val = []
avgDT_T1T2_val = []
avgDT_T3T4_val = []
avgRF_All_val = []
avgRF_T0_val = []
avgRF_T1T2_val = []
avgRF_T3T4_val = []

for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]
    
    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences
    docsTrain = []
    for review in train1["abstract"]:
        result = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        sentences += result
        docsTrain.append(result)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    print "Parsing sentences from test set"
    # read in the test set as a list of a list of words
    docs = []
    for review in test1["abstract"]:
        docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))
        
    print "Parsing sentences from validation set"
    docs_val = []
    for k in range(len(df_val)):
        review = df_val["abstract"].iloc[k]
        sents = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        docs_val.append(sents)

    print "Scoring test set"
    tupT0 = buildModels(train1, test1, docs, docsTrain, df_val, docs_val, basemodel, 'T0')
    scoreDT_T0 = tupT0[0]
    scoreRF_T0 = tupT0[1]
    print scoreDT_T0
    print scoreRF_T0
    print "end T0"
    avgDT_T0.append(scoreDT_T0)
    avgRF_T0.append(scoreRF_T0)
    avgDT_All.append(scoreDT_T0)
    avgRF_All.append(scoreRF_T0)
    modelsT0 = tupT0[2]
    ps_testT0 = tupT0[3]
    ps_trainT0 = tupT0[4]
    clfT0 = tupT0[5]
    forestT0 = tupT0[6]
    #(scoreDT, scoreRF, models, probssentencesArticle, probssentences_train, clf, forest)
    
    scoreDT_T0_val = tupT0[7]
    scoreRF_T0_val = tupT0[8]
    avgDT_T0_val.append(scoreDT_T0_val)
    avgRF_T0_val.append(scoreRF_T0_val)
    avgDT_All_val.append(scoreDT_T0_val)
    avgRF_All_val.append(scoreRF_T0_val)

    tupT1T2 = buildModels(train1, test1, docs, docsTrain, df_val, docs_val, basemodel, 'T1/T2')
    scoreDT_T1T2 = tupT1T2[0]
    scoreRF_T1T2 = tupT1T2[1]
    print scoreDT_T1T2
    print scoreRF_T1T2
    print "end T1T2"
    avgDT_T1T2.append(scoreDT_T1T2)
    avgRF_T1T2.append(scoreRF_T1T2)
    avgDT_All.append(scoreDT_T1T2)
    avgRF_All.append(scoreRF_T1T2)
    modelsT1T2 = tupT1T2[2]
    ps_testT1T2 = tupT1T2[3]
    ps_trainT1T2 = tupT1T2[4]
    clfT1T2 = tupT1T2[5]
    forestT1T2 = tupT1T2[6]
    
    scoreDT_T1T2_val = tupT1T2[7]
    scoreRF_T1T2_val = tupT1T2[8]
    avgDT_T1T2_val.append(scoreDT_T1T2_val)
    avgRF_T1T2_val.append(scoreRF_T1T2_val)
    avgDT_All_val.append(scoreDT_T1T2_val)
    avgRF_All_val.append(scoreRF_T1T2_val)
    

    tupT3T4 = buildModels(train1, test1, docs, docsTrain, df_val, docs_val, basemodel, 'T3/T4')
    scoreDT_T3T4 = tupT3T4[0]
    scoreRF_T3T4 = tupT3T4[1]
    print scoreDT_T3T4
    print scoreRF_T3T4
    print "end T3T4"
    avgDT_T3T4.append(scoreDT_T3T4)
    avgRF_T3T4.append(scoreRF_T3T4)
    avgDT_All.append(scoreDT_T3T4)
    avgRF_All.append(scoreRF_T3T4)
    modelsT3T4 = tupT3T4[2]
    ps_testT3T4 = tupT3T4[3]
    ps_trainT3T4 = tupT3T4[4]
    clfT3T4 = tupT3T4[5]
    forestT3T4 = tupT3T4[6]
    
    scoreDT_T3T4_val = tupT3T4[7]
    scoreRF_T3T4_val = tupT3T4[8]
    avgDT_T3T4_val.append(scoreDT_T3T4_val)
    avgRF_T3T4_val.append(scoreRF_T3T4_val)
    
    avgDT_All_val.append(scoreDT_T3T4_val)
    avgRF_All_val.append(scoreRF_T3T4_val)

print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE"
print "Average for T0 model:"
avgT0 = sum(avgDT_T0) / float(len(avgDT_T0))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgDT_T1T2) / float(len(avgDT_T1T2))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgDT_T3T4) / float(len(avgDT_T3T4))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgDT_All) / float(len(avgDT_All))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST"
print "Average for T0 model:"
avgT0 = sum(avgRF_T0) / float(len(avgRF_T0))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgRF_T1T2) / float(len(avgRF_T1T2))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgRF_T3T4) / float(len(avgRF_T3T4))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgRF_All) / float(len(avgRF_All))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE VALIDATION"
print "Average for T0 model:"
avgT0 = sum(avgDT_T0_val) / float(len(avgDT_T0_val))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgDT_T1T2_val) / float(len(avgDT_T1T2_val))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgDT_T3T4_val) / float(len(avgDT_T3T4_val))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgDT_All_val) / float(len(avgDT_All_val))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST VALIDATION"
print "Average for T0 model:"
avgT0 = sum(avgRF_T0_val) / float(len(avgRF_T0_val))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgRF_T1T2_val) / float(len(avgRF_T1T2_val))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgRF_T3T4_val) / float(len(avgRF_T3T4_val))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgRF_All_val) / float(len(avgRF_All_val))


    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

In [29]:
pickle.dump(ps_testT0, open('./pickled_objects/probssentencesJournalsDatasetT0.p', 'wb'))
pickle.dump(ps_trainT0, open('./pickled_objects/probssentencesJournalsDataset_trainT0.p', 'wb'))
pickle.dump(ps_testT1T2, open('./pickled_objects/probssentencesJournalsDatasetT1T2.p', 'wb'))
pickle.dump(ps_trainT1T2, open('./pickled_objects/probssentencesJournalsDataset_trainT1T2.p', 'wb'))
pickle.dump(ps_testT3T4, open('./pickled_objects/probssentencesJournalsDatasetT3T4.p', 'wb'))
pickle.dump(ps_trainT3T4, open('./pickled_objects/probssentencesJournalsDataset_trainT3T4.p', 'wb'))

In [28]:
print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE"
print "Average for T0 model:"
avgT0 = sum(avgDT_T0) / float(len(avgDT_T0))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgDT_T1T2) / float(len(avgDT_T1T2))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgDT_T3T4) / float(len(avgDT_T3T4))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgDT_All) / float(len(avgDT_All))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST"
print "Average for T0 model:"
avgT0 = sum(avgRF_T0) / float(len(avgRF_T0))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgRF_T1T2) / float(len(avgRF_T1T2))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgRF_T3T4) / float(len(avgRF_T3T4))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgRF_All) / float(len(avgRF_All))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "DECISION TREE VALIDATION"
print "Average for T0 model:"
avgT0 = sum(avgDT_T0_val) / float(len(avgDT_T0_val))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgDT_T1T2_val) / float(len(avgDT_T1T2_val))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgDT_T3T4_val) / float(len(avgDT_T3T4_val))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgDT_All_val) / float(len(avgDT_All_val))

print ""
print "---------------------------------------------------------------------------------"
print ""
print "RANDOM FOREST VALIDATION"
print "Average for T0 model:"
avgT0 = sum(avgRF_T0_val) / float(len(avgRF_T0_val))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgRF_T1T2_val) / float(len(avgRF_T1T2_val))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgRF_T3T4_val) / float(len(avgRF_T3T4_val))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgRF_All_val) / float(len(avgRF_All_val))


---------------------------------------------------------------------------------

DECISION TREE
Average for T0 model:
0.937399123722
Average for T1/T2 model:
0.84274531664
Average for T3/T4 model:
0.86948293702

Average across all models:
0.883209125794

---------------------------------------------------------------------------------

RANDOM FOREST
Average for T0 model:
0.935533882479
Average for T1/T2 model:
0.822510517119
Average for T3/T4 model:
0.857454361667

Average across all models:
0.871832920422

---------------------------------------------------------------------------------

DECISION TREE VALIDATION
Average for T0 model:
0.586149691358
Average for T1/T2 model:
0.708134512546
Average for T3/T4 model:
0.744353518822

Average across all models:
0.679545907575

---------------------------------------------------------------------------------

RANDOM FOREST VALIDATION
Average for T0 model:
0.586535493827
Average for T1/T2 model:
0.703568490333
Average for T3/T4 model:
0.7377

In [1]:
print(b-a)

NameError: name 'b' is not defined

### Kaggle Word2Vec Utility

In [12]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences